# Assignment B3
Problem Statement: Given a bank customer, build a neural network-based classifier that can determine
whetherthey will leave or not in the next 6 months.
Dataset Description: The case study is from an open-source dataset from
Kaggle.The dataset contains 10,000 sample points with 14 distinct features
such as CustomerId, CreditScore, Geography, Gender, Age, Tenure, Balance,
etc.
Link to the Kaggle project:
https://www.kaggle.com/barelydedicated/bank-customer-churnmodeling Perform following steps:
1. Read the dataset.
2. Distinguish the feature and target set and divide the data set into training and test
sets.
3. Normalize the train and test data.
4. Initialize and buil

 the model. Identify the points of improvement and i plement the
same.
Print the accuracy score and confusion matrix

In [13]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
!pip install tensorflow

  Using cached tensorflow-2.14.0-cp311-cp311-win_amd64.whl (2.1 kB)
  Using cached tensorflow_intel-2.14.0-cp311-cp311-win_amd64.whl (284.2 MB)
  Using cached absl_py-2.0.0-py3-none-any.whl (130 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached gast-0.5.4-py3-none-any.whl (19 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached grpcio-1.59.0-cp311-cp311-win_amd64.whl (3.7 MB)
  Using cached tensorboard-2.14.1-py3-none-any.whl (5.5 MB)
  Using cached google_auth-2.23.3-py2.py3-none-any.whl (182 kB)
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl (18 kB)
  Using cached cachetools-5.3.1-py3-none-any.whl (9.3 kB)
  Using cached pyasn1_modules-0.3.0-py2.py3-none-any.whl (181 kB)
  Using cached rsa-4.9-py3-none-any.whl (34 kB)



[notice] A new release of pip is available: 23.1.2 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
df = pd.read_csv("bank_data.csv")
df.sample(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
2574,2575,15688691,Lei,665,Germany,Female,51,9,110610.41,2,0,1,1112.76,1
2233,2234,15791268,Neumann,565,Spain,Male,38,0,122447.76,1,0,0,67339.34,0
1873,1874,15766686,Nebechi,659,Germany,Female,39,1,104502.11,1,1,0,20652.69,0
331,332,15601274,Hsieh,667,Spain,Female,40,1,146502.07,1,1,0,19162.89,0
751,752,15634245,Muecke,758,Germany,Female,47,9,95523.16,1,1,0,73294.48,0


## Preprocessing

In [36]:
#Remove useless columns
df.drop('Surname',axis='columns',inplace=True)
df.drop('CustomerId',axis='columns',inplace=True)

In [37]:
df['Gender'].replace({'Female':1,'Male':0},inplace=True)

In [38]:
df = pd.get_dummies(data=df, columns=['Geography'])
df.columns

Index(['RowNumber', 'CreditScore', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited', 'Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype='object')

In [39]:
df.head(5)

,RowNumber,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,1,619,1,42,2,0.00,1,1,1,101348.88,1,True,False,False
1,2,608,1,41,1,83807.86,1,0,1,112542.58,0,False,False,True
2,3,502,1,42,8,159660.80,3,1,0,113931.57,1,True,False,False
3,4,699,1,39,1,0.00,2,0,0,93826.63,0,True,False,False
4,5,850,1,43,2,125510.82,1,1,1,79084.10,0,False,False,True


## Scaling of data

In [40]:
cols_to_scale = ['Tenure','Balance','Age','EstimatedSalary','CreditScore','NumOfProducts']

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

In [41]:
df.head(5)

,RowNumber,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,1,0.538,1,0.324324,0.2,0.000000,0.000000,1,1,0.506735,1,True,False,False
1,2,0.516,1,0.310811,0.1,0.334031,0.000000,0,1,0.562709,0,False,False,True
2,3,0.304,1,0.324324,0.8,0.636357,0.666667,1,0,0.569654,1,True,False,False
3,4,0.698,1,0.283784,0.1,0.000000,0.333333,0,0,0.469120,0,True,False,False
4,5,1.000,1,0.337838,0.2,0.500246,0.000000,1,1,0.395400,0,False,False,True


In [43]:
X = df.drop('Exited',axis='columns')
y = df['Exited']


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=5)
X_train = np.asarray(X_train).astype(np.int32)
X_test = np.asarray(X_test).astype(np.int32)
y_train = np.asarray(y_train).astype(np.int32)
y_test = np.asarray(y_test).astype(np.int32)

## Building neural network

In [44]:
import tensorflow as tf
from tensorflow import keras


model = keras.Sequential([
    keras.layers.Dense(10, input_shape=(13,), activation='relu'),
    keras.layers.Dense(8, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100)

Epoch 1/100
250/250 [==============================] - 3s 4ms/step - loss: 36.0485 - accuracy: 0.5911
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.6311 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5993 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5725 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5517 - accuracy: 0.7960
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5363 - accuracy: 0.7960
Epoch 7/100
250/250 [==============================] - 1s 5ms/step - loss: 0.5254 - accuracy: 0.7960
Epoch 8/100
250/250 [==============================] - 1s 5ms/step - loss: 0.5180 - accuracy: 0.7960
Epoch 9/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5131 - accuracy: 0.7960
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5100 - accur

In [45]:
model.evaluate(X_test, y_test)

63/63 [==============================] - 1s 5ms/step - loss: 0.5037 - accuracy: 0.7975


[0.5037492513656616, 0.7975000143051147]

In [46]:
yp = model.predict(X_test)

63/63 [==============================] - 1s 5ms/step


In [47]:
y_pred = []
for element in yp:
    if element > 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [48]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.80      1.00      0.89      1595
           1       0.50      0.00      0.00       405

    accuracy                           0.80      2000
   macro avg       0.65      0.50      0.45      2000
weighted avg       0.74      0.80      0.71      2000

